this notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


# 1.- Segmenting and Clustering Neighborhoods in Toronto
First we import the relevant libraries

In [3]:
import pandas as pd
import numpy  as np
import urllib.request
from bs4 import BeautifulSoup

then we extract the table from wikipedia using Beautiful Soup

In [4]:
url   = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page  = urllib.request.urlopen(url)
soup  = BeautifulSoup(page)
table = soup.find('table', class_ = 'wikitable sortable')

once we have the html table, we convert it into arrays for each column

In [5]:
A = []
B = []
C = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        A.append(cells[0].find(text = True)[: - 1])
        B.append(cells[1].find(text = True)[: - 1])
        C.append(cells[2].find(text = True)[: - 1])
A = np.array(A)
B = np.array(B)
C = np.array(C)

and create a dataframe with the information, we filter by the Borough column in order to remove the "Not assigned" cases.

In [6]:
col_names = ['Postal Code', 'Borough', 'Neighbourhood']
df = pd.DataFrame(columns = col_names)
BB = np.array(B) != 'Not assigned'
df['Postal Code']   = A[BB]
df['Borough']       = B[BB]
df['Neighbourhood'] = C[BB]
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


finally we print the shape of the Dataframe

In [7]:
df.shape

(103, 3)

# 2.- Add longitude - latitude to the Dataframe

Tried using geocoder, it didn't work. First we open the csv file

In [8]:
coords = pd.read_csv("Geospatial_Coordinates.csv")

Then we merge the two dataframes by the 'Postal  Code' columns, using an outher merger in order to get the same Dataframe than the example

In [9]:
df = pd.merge(df, coords, on='Postal Code', how='outer')

# 3.- Explore and cluster the neighbourhoods in Toronto

Is not clear to me if I should use the Foursquare information in order to produce a clustering, or the objetive is only to plot using folium the points in the dataframe. Given that is only 3 over 15 points, I assume the later.

In [10]:
import folium
latitude    = 43.6532
longitude   = - 79.3832
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

We first define the map of toronto with the coordinates, then go through all the entries of the Dataframe, and check if 'Toronto' is in the name in order to use a different color than the rest of the neighbourhoods

In [11]:
# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    clr = 'blue'
    if 'Toronto' in borough:
        clr = 'red'
    folium.CircleMarker(
        [lat, lng],
        radius       = 5,
        popup        = label,
        color        = clr,
        fill         = True,
        fill_color   = '#3186cc',
        fill_opacity = 0.7,
        parse_html   = False).add_to(map_toronto)  
    
map_toronto